In [15]:
import pandas as pd
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile
import numpy as np
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

import warnings
warnings.filterwarnings('ignore')




### Quandle data

In [ ]:
# get data downloaded from NASDAQ via Quandl
df = (pd.read_csv('../Findata/wiki_prices.csv',
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())

# stock names
df_wiki = pd.read_csv('../Findata/wiki_stocks.csv')

# data path
data_path = Path('..') / 'Findata' / 'data.h5'

# store hdf5 file for easy access
with pd.HDFStore(data_path) as store:
    store.put('quandl/wiki/prices', df)
    store.put('quandl/wiki/stocks', df_wiki)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   open         float64
 1   high         float64
 2   low          float64
 3   close        float64
 4   volume       float64
 5   ex-dividend  float64
 6   split_ratio  float64
 7   adj_open     float64
 8   adj_high     float64
 9   adj_low      float64
 10  adj_close    float64
 11  adj_volume   float64
dtypes: float64(12)
memory usage: 1.4+ GB


### S&P500 data

In [21]:

# S&P500 data
sp500 = web.DataReader(name='SP500', data_source='fred', start=2009).squeeze().to_frame('close')

# get S&P500 constituents
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500cons = pd.read_html(url, header=0)[0]

# rename columns
sp500cons.columns = ['ticker','name', 'sec_filings', 'gics_sector', 'location', 'first_added', 'cik', 'founded']

# store in hdf5
with pd.HDFStore(data_path) as store:
    store.put('sp500/fred', sp500)
    store.put('sp500/stocks', sp500cons)

### Metadata of companies

In [70]:

# get NYSE, NASDAQ, and AMEX stock data
amex = pd.read_csv('../Findata/amex.csv')
nasdaq = pd.read_csv('../Findata/nasdaq.csv')
nyse = pd.read_csv('../Findata/nyse.csv')

print(amex.shape, nasdaq.shape, nyse.shape)

# concat all
df_all = pd.concat([amex, nasdaq, nyse])
# rename columns
df_all.rename(columns=str.lower,inplace=True)
df_all.columns = df_all.columns.str.replace(' ','')
df_all.rename(columns={'symbol':'ticker'},inplace=True)

# set index
df_all.set_index('ticker',inplace=True)

# drop columns
df_all.drop(columns=['%change','country','volume','netchange'],inplace=True)

# drop duplicates
df_all = df_all[~df_all.index.duplicated(keep='first')]

# store data in csv
df_all.to_csv('../Findata/us_equity_metadata.csv')

(309, 11) (4158, 11) (2843, 11)


In [74]:
with pd.HDFStore(data_path) as store:
    store.put('us_equities/stocks', df_all)